# Algoritmo que permite resolver problemas QUBO densos empleando Tensor Network

En este notebook vamos a crear e implementar un algoritmo en tensor networks basado en el método de las señales y evolución en tiempo imaginario para resolver problemas QUBO. Utiliza un método de eliminación de capas de evolución para reducir la complejidad computacional.

Versiones:

- alpha 0: Implementación básica.
- alpha 1: Añade el método de eliminación de capas.
- alpha 2: Versión que va comprimiendo el MPS de derecha a izquierda

In [3]:
# Librerias
import numpy as np
from time import time
import tensorkrowch as tk
import torch
from quantum_sim.TensorNetwork.QUBO.qubo_core.qubo_solvers import  qubo_dimod_solver, recocido_simulado, random_qubo_solver
from quantum_sim.TensorNetwork.QUBO.qubo_core.qubo_auxiliar_functions import evaluar_qubo, generar_matriz_qubo, matrix_QUBO_to_dict
from quantum_sim.TensorNetwork.main.tensor_compresor import compresor

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
# Funciones de la tensor network

## Definimos los nodos de la Tensor Network

In [4]:
def node_qubit(tn:tk.TensorNetwork, value,row):
    if value == 0:
        node = tk.Node(tensor = torch.tensor([1,0], dtype=torch.float64), network=tn, name = 'qudit_0', axes_names = ['right'])
    elif value ==1:
        node = tk.Node(tensor = torch.tensor([0,1], dtype=torch.float64), network=tn, name = 'qudit_1', axes_names = ['right'])
    elif value ==2:
        node = tk.Node(tensor = torch.tensor([1,1], dtype=torch.float64), network=tn, name = f'qudit_+{row}', axes_names = ['left'])
    elif value ==3:
        node = tk.Node(tensor = torch.tensor([1,-1], dtype=torch.float64), network=tn, name = 'qudit_-', axes_names = ['left'])
    return node

def node_Ai0(tn:tk.TensorNetwork, Q_element:float, qudit:int):
    """  
    Nodos de superposicion con el termino diagonal.
    """
    node = tk.Node(tensor = torch.tensor([1,Q_element], dtype=torch.float64), network=tn, name = f'qubit_({qudit})', axes_names = ['right'])
    return node

def node_A0i(tn:tk.TensorNetwork, qudit:int):
    """
    Nodos de control.
    """
    tensor = torch.zeros((2,2,2), dtype = torch.float64)
    for i in range(0,2):
        j=i; k=i
        tensor[i,j,k]= 1
    node = tk.Node(tensor = tensor, network=tn, name = f'A_({qudit},{qudit+1})', axes_names = ['left','right','down'])
    return node

def node_Aii(tn:tk.TensorNetwork, Q_element:float, row:int, column:int):
    """
    Nodos de evolucion.
    """
    tensor = torch.zeros((2,2,2,2), dtype = torch.float64)
    for k in range(0,2):
        for i in range(0,2):
            l=k; j=i
            if k*i == 1:
                tensor[i,j,k,l] = Q_element
            else:
                tensor[i,j,k,l] = 1
    node = tk.Node(tensor = tensor, network = tn, name = f'A_({row},{column})', axes_names=['left','right','up','down'])
    return node

def node_Afi(tn:tk.TensorNetwork, Q_element:float, column,max_element):
    """
    Nodo final de evolucion.
    """
    tensor = torch.zeros((2,2,2), dtype = torch.float64)
    for k in range(2):
        for i in range(2):
            j=i
            if k*i == 1:
                tensor[i,j,k] = Q_element
            else:
                tensor[i,j,k] = 1

    node = tk.Node(tensor=tensor, network=tn, name = f'A_({max_element-1},{column})',axes_names=['left','right','up'])
    return node



## Generación de la Tensor Network

In [5]:
def tensor_network_generator(Q_matrix:np.array, tau:float)->list[list[tk.Node]]:
    """   
    Args:

    Return:

    """
    # Inicializamos la tensor network
    n_variables  = len(Q_matrix[0])
    network      = tk.TensorNetwork('QUBO_TN')
    tensor_network_matrix = [[None] * min(row+1+2, n_variables+2-1) for row in range(n_variables)]

    # Exponencial de Q elementwise
    Q_exponential = np.exp(-tau*Q_matrix)

    # Capa de superposicion inicial
    for row in range(n_variables):
        tensor_network_matrix[row][0] = node_Ai0(network, Q_exponential[row][row], row)

    # Capas de evolucion
    for column in range(1, n_variables):
        # Primer nodo
        tensor_network_matrix[column-1][column] = node_A0i(network, column-1)

        # Nodos intermedios
        for row in range(column,n_variables-1):
            tensor_network_matrix[row][column] = node_Aii(network, Q_exponential[row][column-1], row, column)
        
        # Ultimo nodo
        tensor_network_matrix[n_variables-1][column] = node_Afi(network, Q_exponential[n_variables-1][column-1], column, n_variables)

    # Capa de traceado
    tensor_network_matrix[0][2] = node_qubit(network, 3, row)
    for row in range(1, n_variables):
        tensor_network_matrix[row][min(row+2, n_variables+2-2)] = node_qubit(network, 2, row)

    return  tensor_network_matrix, network

## Conexión y contracción de toda la Tensor Network

In [6]:
def mps_reshape(mps: list[tk.Node], network: tk.TensorNetwork) -> list[tk.Node]:
     # Primer nodo
     aux_mps = mps[0].tensor
     aux_mps = torch.moveaxis(aux_mps, (0, 1, 2), (1, 0, 2))
     aux_mps = torch.reshape(aux_mps, (1,aux_mps.shape[0], aux_mps.shape[1]*aux_mps.shape[2]))
     network._remove_node(mps[0])
     mps[0] = tk.Node(tensor = aux_mps, name = f'mps_({0})', axes_names = ['up', 'right', 'down'], network = network)

     # Nodos intermedios
     for column in range(1, len(mps)-1):
          aux_mps = mps[column].tensor
          aux_mps = torch.moveaxis(aux_mps, (0, 3, 2, 1, 4), (0, 1, 2, 3, 4))
          aux_mps = torch.reshape(aux_mps, (aux_mps.shape[0]*aux_mps.shape[1],aux_mps.shape[2], aux_mps.shape[3]*aux_mps.shape[4]))
          network._remove_node(mps[column])
          mps[column] = tk.Node(tensor = aux_mps, name = f'mps_({column})', axes_names = ['up', 'right', 'down'], network = network)

     # Final
     aux_mps = mps[-1].tensor
     aux_mps = torch.moveaxis(aux_mps, (0, 1, 2), (0, 2, 1))
     aux_mps = torch.reshape(aux_mps, (aux_mps.shape[0]*aux_mps.shape[1], aux_mps.shape[2], 1))
     network._remove_node(mps[-1])
     mps[-1] = tk.Node(tensor = aux_mps, name = f'mps_({len(mps)-1})', axes_names = ['up', 'right', 'down'], network = network)
     return mps

def mps_desreshape(mps: list[tk.Node], network: tk.TensorNetwork) -> list[tk.Node]:
     # Primer nodo
     aux_mps = mps[0].tensor
     aux_mps = torch.reshape(aux_mps, (2,aux_mps.shape[2]))
     network._remove_node(mps[0])
     mps[0] = tk.Node(tensor = aux_mps, name = f'mps_({0})', axes_names = ['right', 'down'], network = network)
     
     # Nodos intermedios
     for column in range(1, len(mps)-1):
          aux_mps = mps[column].tensor
          aux_mps = torch.moveaxis(aux_mps, (0, 1, 2), (1, 0, 2))
          #aux_mps = torch.reshape(aux_mps, (2, aux_mps.shape[0], aux_mps.shape[2]))
          #aux_mps = torch.moveaxis(aux_mps, (0, 1, 2), (1, 0, 2))
          network._remove_node(mps[column])
          mps[column] = tk.Node(tensor = aux_mps, name = f'mps_({column})', axes_names = ['right','up', 'down'], network = network)   

     # Nodo final
     aux_mps = mps[-1].tensor
     aux_mps = torch.reshape(aux_mps, (aux_mps.shape[0], aux_mps.shape[1]))
     aux_mps = torch.moveaxis(aux_mps, (0, 1), (1, 0))
     network._remove_node(mps[-1])
     mps[-1] = tk.Node(tensor = aux_mps, name = f'mps_({len(mps)-1})', axes_names = ['right', 'up'], network = network)
     return mps 
     

In [7]:
def contraccion_arriba(mps: list[tk.Node], matrix_nodes:list[list[tk.Node]]):
    mps[0]['right'] ^ matrix_nodes[0][2]['left']
    mps[0]['down'] ^ mps[1]['up']
    mps[0] = tk.contract_between_(mps[0], matrix_nodes[0][2])  
    mps[1] = tk.contract_between_(mps[0], mps[1])  
    matrix_nodes.pop(0)
    return mps[1:]


In [8]:
def mps_contraction(tensor_network_matrix:list[list[tk.Node]], network, max_rank : int)->np.array:
    """
    Funcion que contrae la tensor network de derecha a izquierda en formato de mps y va realizando la compresión según un cierto error relativo y un rango maximo.

    Args:
        matrix_nodes (list[list[tk.Node]]): _description_
        etol (float): _description_
        max_rank (int): _description_

    Returns:
        np.array: _description_
    """
    matrix_nodes = tensor_network_matrix.copy()
    # Creamos el mps que se va a ir contrayendo de derecha a izquierda
    mps = [row[0] for row in matrix_nodes]
    # contraccion de la primera capa
    for row in range(len(mps)):
        
        mps[row]['right'] ^ matrix_nodes[row][1]['left']
        mps[row] = tk.contract_between_(mps[row], matrix_nodes[row][1])
    mps = contraccion_arriba(mps, matrix_nodes)
    # contraccion del resto de capas
    for i in range(len(matrix_nodes[-1])-3):
        # conexion y contracción vertical con la siguiente capa
        for row in range(len(mps)):
            if matrix_nodes[row][2] is not None:
                mps[row]['right'] ^ matrix_nodes[row][2]['left']
                mps[row] = tk.contract_between_(mps[row], matrix_nodes[row][2])
        # Dar forma a los tensores externos de tensor-train
        mps = mps_reshape(mps, network)
        # Comprimir el mps para reducir la dimension de enlace.
        mps = compresor(mps, max_rank)
        # Devolver el tensor a la forma original
        mps = mps_desreshape(mps, network)

        #Eliminamos la primera columna de matrix_nodes
        for row in range(len(mps)):
            matrix_nodes[row].pop(0)
        #Contraemos con el nodo resultante    
        mps = contraccion_arriba(mps, matrix_nodes)
    return np.dot(mps[0].tensor,np.ones(2))



---
# Función general

Esta es la función que se encarga del proceso general. Se encarga del proceso de minimización resolviendo iterativamente cada una de las variables. Su proceso consiste en la creación de la tensor network, su contracción y la determinación de la variable a partir del vector resultante.

In [9]:
def qubo_solver(Q_matrix:np.array, tau:float, max_rank:int)->np.array:
    """
    Args:
    - Q_matrix: matriz de pesos del problema QUBO.
    - tau: factor de amortiguamiento de la evolucion en tiempo imaginario.
    
    Return:
    - solution: vector de solucion del problema.
    """
    # Determinamos el tamaño del problema
    n_variables = Q_matrix.shape[0]
    solution = np.zeros(n_variables, dtype=int)
    # Matrix QUBO auxiliar para las iteraciones
    Q_matrix_aux = Q_matrix.copy()
    # Generamos todos los tensores del problema
    for variable  in range(n_variables-1):
        
        tensor_network_matrix, network = tensor_network_generator(Q_matrix_aux, tau)
        #[[print(_,__,tensor_network_matrix[_][__]) for __ in range(len(tensor_network_matrix[_]))] for _ in range(len(tensor_network_matrix))]
        result_vector = mps_contraction(tensor_network_matrix, network, max_rank)
        if result_vector < 0:
            solution[variable] = 1

        if solution[variable] == 1:
            for column in range(Q_matrix_aux.shape[1]):
                Q_matrix_aux[column][column] += Q_matrix_aux[column][0]
        # Borramos la primera fila y columna
        Q_matrix_aux = Q_matrix_aux[1:,1:]
    if Q_matrix_aux[0][0] < 0:
        solution[-1] = 1
    return solution


---
# Pruebas

#### Comparación con un recocido simulado obtenido de ChatGPT

In [10]:
def Q_order(Q_matrix:np.array):
    Q_matrix_aux = np.zeros(Q_matrix.shape, dtype=float)
    list_of_sums = np.zeros(Q_matrix.shape[0])

    Q_aux = Q_matrix + Q_matrix.T - np.diagonal(Q_matrix)
    for row in range(Q_matrix.shape[0]):
        list_of_sums[row] = sum(Q_aux[row])

    indexes = np.argsort(list_of_sums)#[::-1]

    for i in range(Q_matrix.shape[0]):
        for j in range(Q_matrix.shape[1]):
            Q_matrix_aux[i,j] = Q_matrix[indexes[i], indexes[j]]

    return Q_matrix_aux

In [16]:
#np.random.seed(31)
n_variables =5
n_diagonals = 5
tau = 300
max_rank = 400
#number_layers = 5
#for k in range(20):
Q_matrix = generar_matriz_qubo(n_variables, n_diagonals)
Q_matrix/= np.linalg.norm(Q_matrix)
Q_matrix_dict = matrix_QUBO_to_dict(Q_matrix)
#print(Q_matrix)
# Inicial RS
x_inicial = np.random.randint(2, size=n_variables)

# TN
solution = qubo_solver(Q_matrix, tau, max_rank)
print('Solution TN:     ', str(solution))
print('Coste de TN:     ', evaluar_qubo(Q_matrix, solution))


# RS
solution = recocido_simulado(Q_matrix, x_inicial, 10.0, 0.99, int(1e4))
print('Solution RS:     ', str(solution))
print('Coste de RS:     ', evaluar_qubo(Q_matrix, solution))
'''
# RS iterativo
solution = qubo_solver_rs(Q_matrix, number_layers)
#print('Solution RS iter: ', str(solution))
print('Coste de RS iter: ', evaluar_qubo(Q_matrix, solution))
'''
# Dimod

solution_dimod = qubo_dimod_solver(Q_matrix_dict, "neal")
print('Coste de dimod iter: ', evaluar_qubo(Q_matrix, solution_dimod))

best_c = random_qubo_solver(Q_matrix)
print('Coste de Random: ', evaluar_qubo(Q_matrix, best_c))


Solution TN:      [1 1 1 1 1]
Coste de TN:      -1.7469565759717753
Solution RS:      [1 1 1 1 1]
Coste de RS:      -1.7469565759717753
Coste de dimod iter:  -1.7469565759717753
